# Occlusion

### What it this?

This script takes images from a given directory and returns their occluded version.

### How does it work?

Very good question. First check the following requirements:
1. Make sure this script is located in a directory of which the parent directory contains a folder named `images` and where all the images you wish to work on are located.
2. Make sure the input images are 4-channel, with the background gray & transparent.

Then follow these simple steps for a smooth experience:
1. Make sure you select the options you wish for in the preamble of this script. In particular, set the following to `True`/`False` to determine the output(s) you will get:

- **deletion**: if set to `True`, images will be occluded with blobs the same color as the background;
- **partialViewing**: if set to `True`, images will be occluded fully with openings revealing parts of the objects;
- **blob**: if set to `True`, images will be occluded with blobs of a color different than the background.

2. Select the color you wish for the partial viewing and blob manipulations. Default is black.
3. Select your preference between many small occluders and few large occluders (choose only one).

## Preamble

In [1]:
import cv2 as cv
import numpy as np
import glob
import os
import time
from numpy.random import randint
print("All libraries are loaded.")

All libraries are loaded.


In [2]:
np.random.seed()

A note on the naming: in variable names, I always use *high* and *low* with reference to high and low **levels of occlusion**. Consequently:
- In the _high_ condition, only a small portion of the object is visible.
- In the _low_ condition, a large portion of the object remains visible.

In [3]:
# decide if you prefer many small occluders OR few large occluders
manySmall = False
fewLarge = True

if manySmall:
    sizeOccluder = [20, 90]
    nOccluders_low = 15
    nOccluders_high = 50
    
elif fewLarge:
    sizeOccluder = [70, 300]
    nOccluders_low = 5
    nOccluders_high = 5

# decide on the portion of object **still visible** in high and low **levels of occlusion**
portionHigh = 30
portionLow = 70

In [4]:
# decide on the colors to use, black by default
colPartialViewing = 0 # color of the occluding window (grayscale)
colBlob = (0,0,0) # color of the occluding blobs

In [5]:
# decide on the manipulations
doDeletion = True
doPartialViewing = True
doBlob = True

doManipulations = (doDeletion, doPartialViewing, doBlob)



### Set the input path

In [6]:
# give the input path
parentPath = os.pardir
inputPath = parentPath + '/images/'

imagePath = glob.glob(inputPath + "*") # take in all the files in there
imagePath.sort() # make sure the files are sorted

# count the number of images to process
nbrIm = len(imagePath)

print('The input path is ', inputPath, ' and contains ', nbrIm, ' images.')

The input path is  ../images/  and contains  53  images.


### Set the output path

In [7]:
# name the output paths
outputPath = parentPath + r'/output/occlusionOuput/'
outputPathDeletion = outputPath + 'deletion/'
outputPathPartialViewing = outputPath + 'partialViewing/'
outputPathBlob = outputPath + 'blob/'

listOutputPaths = [outputPathDeletion, outputPathPartialViewing, outputPathBlob]

# create the necessary output folders
# manipulations = (deletion, partialViewing, blob)
idx = 0

for manipulation in doManipulations:
    if manipulation:
        if not os.path.exists(listOutputPaths[idx]):
            os.makedirs(listOutputPaths[idx])
            idx +=1

## Start occluding

**Deletion & Blobs**: a word on the technique

The way deletion and blobs work so that they only cover a given portion of the object itself (and only of the object, not of the whole image) is the following:
1. When importing the image, an object mask is extracted
2. When patches of the image are deleted/blobs are added, an occluder mask is created
3. The intersection of both masks is measured (in pixel size)
4. The proportion of that intersection to the total object size is calculated. From there:
    - If it falls within the desired boundaries (e.g. 50% of object occluded) the operation is stopped and the image saved
    - It if doesn't, one occluder is added/removed depending on the sign of the difference, and the operation is run again.

In [8]:
# find the BGR color of the background from the upper left-most pixel
colBckgrnd = ((cv.imread(imagePath[0], -1))[0,0,0:3])
colBckgrnd = int(colBckgrnd[0]), int(colBckgrnd[1]), int(colBckgrnd[2])
print('Color background is', colBckgrnd)

Color background is (80, 80, 80)


The following three cells each run one of the occluding manipulations. Only the manipulations that have been set to `True` in the preamble will be applied.

In [9]:
# deletion

if doManipulations[0]:
    for file in imagePath:
        
        # read the image
        img = cv.imread(file, -1)
        
        # extract image name
        basename = os.path.basename(file)
        name = os.path.splitext(basename)[0]

        # creating object masks & measuring pixel size
        # creating a white mask where the object is located
        object_mask = np.zeros((img.shape[0], img.shape[1]), np.uint8)
        object_mask[img[:,:,3]==255] = 255
        # measuring the pixel size of the object
        size_object = object_mask[object_mask==255].shape[0]

        # removing transparency in the background and make image 2D
        img[:,:,3] == 255

        # operate the occlusion for each level
        for level in ('low', 'high'):
            
            # create the write path, one for each level
            writePath = outputPathDeletion + "{}/".format(level)
            if not os.path.exists(writePath): os.makedirs(writePath)

            if level == 'low':
                nOccluders = nOccluders_low
                portionVisible = portionLow
            elif level == 'high':
                nOccluders = nOccluders_high
                portionVisible = portionHigh

            proportion_occluded = 0
                
            while abs(portionVisible - (1 - proportion_occluded)) > 1:
                
                # make a 2D copy of the image
                gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
                
                # create the coordinates for occluders
                points_1 = np.array(randint(0, gray.shape[0], nOccluders))
                points_2 = np.array(randint(0, gray.shape[1], nOccluders))
                points = np.column_stack((points_1,points_2))

                # randomly generate the blob radiuses within the determined range
                radiuses = randint(sizeOccluder[0], sizeOccluder[1], nOccluders)

                # preparing the background for occluder mask
                # create the black figure that will hold the occluder mask
                occluder_mask = np.zeros((gray.shape[0], gray.shape[1]), np.uint8)

                # drawing occluders on the image & on their mask
                for point, radius in zip(points, radiuses):
                    cv.circle(gray, point, radius, tuple(colBckgrnd[0:3]), -1)
                    # here I draw the occluder mask
                    cv.circle(occluder_mask, point, radius, color = (255, 255, 255), thickness = -1)

                # calculate the intersection between masks
                intersection = cv.bitwise_and(object_mask, occluder_mask) # create the intersection
                intersectionSize = intersection[intersection==255].shape[0] # measure the size of intersection
                proportion_occluded = ((intersectionSize/size_object)*100)

                # write the image only if occlusion is as required
                if abs(portionVisible - (100 - proportion_occluded)) < 1:
                    cv.imwrite(writePath + '_' + str(level) + '/deletion{}'.format(level) + name + '.png', gray)
                    print('Object occluded successfully')
                    break
                elif portionVisible > (100 - proportion_occluded):
                    #nOccluders += nOccluders
                    print('Occlusion is too low, the difference is', 
                          abs(portionVisible - (100 - proportion_occluded)))
                    continue
                elif portionVisible < (100 - proportion_occluded):
                    #nOccluders -= nOccluders
                    print('Occlusion is too high, the difference is', 
                          abs(portionVisible - (100 - proportion_occluded)))
                    continue

else:
    print('Deletion is off. Change parameters in the preamble to run it.')

Occlusion is too low, the difference is 27.086454310620397
Occlusion is too high, the difference is 21.019360858409144
Occlusion is too low, the difference is 3.0516894314421705
Occlusion is too low, the difference is 19.1978332426198
Occlusion is too low, the difference is 67.47298032812378
Occlusion is too low, the difference is 69.35031836128174
Occlusion is too low, the difference is 22.45833657310215
Occlusion is too low, the difference is 42.80235682381705
Occlusion is too high, the difference is 9.79507382353195
Occlusion is too low, the difference is 42.49306689474639
Occlusion is too high, the difference is 10.023153547762831
Occlusion is too low, the difference is 28.09121461092537
Occlusion is too low, the difference is 19.399130878020543
Occlusion is too high, the difference is 4.207984518224777
Occlusion is too low, the difference is 68.96241004241936
Occlusion is too low, the difference is 29.446733881070244
Occlusion is too high, the difference is 30.0
Occlusion is too l

Object occluded successfully
Occlusion is too low, the difference is 17.35686406649257
Occlusion is too low, the difference is 68.90826383623957
Occlusion is too high, the difference is 5.195574649706572
Occlusion is too low, the difference is 45.45896161514054
Occlusion is too high, the difference is 6.9106786847499535
Occlusion is too high, the difference is 27.347597787324858
Occlusion is too high, the difference is 24.730575913290096
Occlusion is too low, the difference is 22.916631567124362
Occlusion is too high, the difference is 28.41069272456687
Occlusion is too high, the difference is 18.12736921910539
Occlusion is too low, the difference is 16.885687810630948
Occlusion is too low, the difference is 57.124364698284325
Occlusion is too low, the difference is 56.15674051610367
Occlusion is too low, the difference is 17.529273018279845
Occlusion is too low, the difference is 1.7479572066380342
Occlusion is too low, the difference is 48.62241316373235
Occlusion is too high, the di

Occlusion is too low, the difference is 18.27900022751649
Occlusion is too low, the difference is 43.389237386377175
Occlusion is too low, the difference is 59.55265923446116
Occlusion is too high, the difference is 19.142046131677887
Occlusion is too low, the difference is 25.90405304384568
Occlusion is too low, the difference is 44.27005124538195
Occlusion is too high, the difference is 4.913597902514596
Occlusion is too low, the difference is 69.62513948927963
Occlusion is too low, the difference is 67.82775917920716
Occlusion is too high, the difference is 28.772494339172923
Occlusion is too low, the difference is 38.48354839059165
Occlusion is too low, the difference is 43.657923532789454
Occlusion is too high, the difference is 27.6164938624717
Occlusion is too high, the difference is 29.596970780381582
Occlusion is too low, the difference is 42.328577155177086
Occlusion is too low, the difference is 59.614413711660774
Occlusion is too high, the difference is 27.508152674402226
O

Occlusion is too low, the difference is 39.01738451303116
Occlusion is too low, the difference is 22.176679876196808
Object occluded successfully
Occlusion is too high, the difference is 50.41074889358133
Occlusion is too high, the difference is 22.353128344566258
Occlusion is too low, the difference is 22.205605854618028
Occlusion is too low, the difference is 13.931618987012229
Occlusion is too high, the difference is 20.325417257238726
Occlusion is too low, the difference is 2.296433426860659
Occlusion is too low, the difference is 22.554453154377953
Occlusion is too low, the difference is 22.415608457956083
Occlusion is too high, the difference is 46.51731219808511
Occlusion is too low, the difference is 27.600879349744005
Occlusion is too high, the difference is 6.361690434178939
Occlusion is too high, the difference is 5.916808886060565
Occlusion is too high, the difference is 59.48135720690752
Occlusion is too high, the difference is 69.78363368140927
Occlusion is too high, the 

Object occluded successfully
Occlusion is too low, the difference is 36.11265682467203
Occlusion is too high, the difference is 16.191353629082585
Occlusion is too low, the difference is 48.856840822499066
Occlusion is too low, the difference is 37.841490540967115
Occlusion is too low, the difference is 45.80440320366468
Occlusion is too high, the difference is 29.568342154912315
Occlusion is too high, the difference is 14.655591363906652
Occlusion is too low, the difference is 39.69512329410729
Occlusion is too high, the difference is 7.290245854102579
Occlusion is too low, the difference is 15.404091940184557
Occlusion is too low, the difference is 38.163031588838564
Occlusion is too high, the difference is 10.05637980017471
Occlusion is too high, the difference is 4.143107789662224
Occlusion is too low, the difference is 19.738289078616056
Occlusion is too low, the difference is 55.86687613328537
Occlusion is too high, the difference is 6.208898905439042
Occlusion is too high, the d

...

Occlusion is too high, the difference is 16.41221581638203
Occlusion is too low, the difference is 32.04041118242141
Occlusion is too low, the difference is 21.543293810507997
Occlusion is too low, the difference is 18.999238551071464
Occlusion is too low, the difference is 37.38958990536278
Occlusion is too high, the difference is 25.76851952572609
Occlusion is too high, the difference is 29.764766670292616
Occlusion is too low, the difference is 19.014195583596212
Occlusion is too low, the difference is 6.938431415207226
Occlusion is too high, the difference is 30.0
Occlusion is too low, the difference is 38.51272707494833
Occlusion is too low, the difference is 13.070814750353534
Occlusion is too low, the difference is 54.447405634722074
Occlusion is too low, the difference is 17.804035679321224
Occlusion is too low, the difference is 45.25290982269118
Occlusion is too low, the difference is 60.61922114652454
Occlusion is too low, the difference is 27.33846404873273
Occlusion is too

Occlusion is too low, the difference is 24.183282980866068
Occlusion is too low, the difference is 63.280966767371595
Occlusion is too low, the difference is 68.2356495468278
Occlusion is too high, the difference is 30.0
Occlusion is too low, the difference is 55.1681772406848
Occlusion is too low, the difference is 52.94461228600201
Occlusion is too high, the difference is 30.0
Occlusion is too low, the difference is 55.9214501510574
Occlusion is too high, the difference is 24.900302114803623
Occlusion is too low, the difference is 8.006042296072508
Occlusion is too low, the difference is 19.091641490433027
Occlusion is too low, the difference is 55.438066465256796
Occlusion is too high, the difference is 30.0
Occlusion is too low, the difference is 20.59415911379658
Occlusion is too high, the difference is 24.73917421953675
Occlusion is too high, the difference is 30.0
Occlusion is too low, the difference is 66.29003021148036
Occlusion is too high, the difference is 2.435045317220542

Occlusion is too high, the difference is 46.13070763903917
Occlusion is too high, the difference is 38.646242309951475
Occlusion is too high, the difference is 14.674931214641234
Occlusion is too high, the difference is 23.99573376201812
Occlusion is too low, the difference is 8.662627136983332
Occlusion is too high, the difference is 30.75679352026463
Occlusion is too high, the difference is 70.0
Occlusion is too low, the difference is 5.571150338516716
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 17.478900670850464
Occlusion is too low, the difference is 24.283859399635205
Occlusion is too high, the difference is 62.32386310940737
Occlusion is too high, the difference is 5.756020651065015
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 15.243762945559098
Occlusion is too high, the difference is 53.11126224997682
Object occluded successfully
Occlusion is too high, the difference is 16.066048294760137
Occl

Occlusion is too high, the difference is 38.380697533802476
Occlusion is too low, the difference is 6.8033345769384255
Occlusion is too low, the difference is 17.49147701689158
Occlusion is too low, the difference is 5.119321763517775
Occlusion is too high, the difference is 4.030181008863906
Occlusion is too high, the difference is 5.6177072210572305
Occlusion is too high, the difference is 18.136522108296568
Occlusion is too high, the difference is 1.0416693468500284
Occlusion is too high, the difference is 70.0
Occlusion is too low, the difference is 27.212180468538932
Occlusion is too high, the difference is 58.214483282088224
Occlusion is too high, the difference is 47.89041695076611
Occlusion is too high, the difference is 54.753830518068725
Occlusion is too low, the difference is 14.204500135081233
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 8.39394836037102
Occlusion is too high, the difference is 7.501125677014315
Occlusion is too high

Occlusion is too high, the difference is 37.27946379179478
Occlusion is too low, the difference is 25.01743191262173
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 14.477750189756982
Object occluded successfully


In [1]:
# partial viewing

if doManipulations[1]:
    for file in imagePath:
        
        # read the image
        img = cv.imread(file, -1)
        
        # extract image name
        basename = os.path.basename(file)
        name = os.path.splitext(basename)[0]
        
        # creating a mask where the object is located
        object_mask = np.zeros((img.shape[0], img.shape[1]), np.uint8)
        object_mask[img[:,:,3]==255] = 255
        # measuring the pixel size of the object
        size_object = object_mask[object_mask==255].shape[0]

        # removing transparency in the background and make image 2D
        img[:,:,3] == 255

        # operate the occlusion for each level
        for level in ('low', 'high'):
            
            # create the write path, one for each level
            writePath = outputPathPartialViewing + "{}/".format(level)
            if not os.path.exists(writePath): os.makedirs(writePath)

            if level == 'low':
                nOccluders = nOccluders_low
                portionVisible = portionLow
            elif level == 'high':
                nOccluders = nOccluders_high
                portionVisible = portionHigh

            proportion_occluded = 0
                
            while abs(portionVisible - (1 - proportion_occluded)) > 1:
                
                # make a 2D copy of the image
                gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
                
                # create the coordinates for occluders
                points_1 = np.array(randint(0, gray.shape[0], nOccluders))
                points_2 = np.array(randint(0, gray.shape[1], nOccluders))
                points = np.column_stack((points_1,points_2))

                # randomly generate the blob radiuses within the determined range
                radiuses = randint(sizeOccluder[0], sizeOccluder[1], nOccluders)
                
                # create the initial full mask
                occluder = np.zeros_like(gray)
                # setting the color of the mask
                occluder[:,:] = colPartialViewing

                # drawing occluders on the image & on their mask
                for point, radius in zip(points, radiuses):
                    cv.circle(occluder, point, radius, (255,255,255), -1)
                occluded = cv.bitwise_and(gray, gray, mask = occluder)
                
                # create the occluder mask
                occluder_inv = cv.bitwise_not(occluder)
                blank = np.ones_like(gray)
                blank[:] = 255
                occluder_mask = cv.bitwise_and(blank, blank, mask = occluder_inv)

                # measuring occluder size
                size_occluder = occluder_mask[occluder_mask==255].shape[0]

                # calculate the intersection between masks
                intersection = cv.bitwise_and(object_mask, occluder_mask) # create the intersection
                intersectionSize = intersection[intersection==255].shape[0] # measure the size of intersection
                proportion_occluded = ((intersectionSize/size_object)*100)

                # write the image only if occlusion is as required
                if abs(portionVisible - (100 - proportion_occluded)) < 1:
                    cv.imwrite(writePath + '_' + str(level) + '/partialViewing{}'.format(level) + name + '.png', occluded)
                    print('Object occluded successfully')
                    break
                elif portionVisible > (100 - proportion_occluded):
                    #nOccluders += nOccluders
                    print('Occlusion is too low, the difference is', 
                          abs(portionVisible - (100 - proportion_occluded)))
                    continue
                elif portionVisible < (100 - proportion_occluded):
                    #nOccluders -= nOccluders
                    print('Occlusion is too high, the difference is', 
                          abs(portionVisible - (100 - proportion_occluded)))
                    continue

else:
    print('Deletion is off. Change parameters in the preamble to run it.')

NameError: name 'doManipulations' is not defined

In [11]:
# blobs

if doManipulations[2]:
    for file in imagePath:
        
        # read the image
        img = cv.imread(file, -1)
        
        # extract image name
        basename = os.path.basename(file)
        name = os.path.splitext(basename)[0]

        # creating object masks & measuring pixel size
        # creating a white mask where the object is located
        object_mask = np.zeros((img.shape[0], img.shape[1]), np.uint8)
        object_mask[img[:,:,3]==255] = 255
        # measuring the pixel size of the object
        size_object = object_mask[object_mask==255].shape[0]

        # removing transparency in the background and make image 2D
        img[:,:,3] == 255

        # operate the occlusion for each level
        for level in ('low', 'high'):
            
            # create the write path, one for each level
            writePath = outputPathBlob + "{}/".format(level)
            if not os.path.exists(writePath): os.makedirs(writePath)

            if level == 'low':
                nOccluders = nOccluders_low
                portionVisible = portionLow
            elif level == 'high':
                nOccluders = nOccluders_high
                portionVisible = portionHigh

            proportion_occluded = 0
                
            while abs(portionVisible - (1 - proportion_occluded)) > 1:
                
                # make a 2D copy of the image
                gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
                
                # create the coordinates for occluders
                points_1 = np.array(randint(0, gray.shape[0], nOccluders))
                points_2 = np.array(randint(0, gray.shape[1], nOccluders))
                points = np.column_stack((points_1,points_2))

                # randomly generate the blob radiuses within the determined range
                radiuses = randint(sizeOccluder[0], sizeOccluder[1], nOccluders)

                # preparing the background for occluder mask
                # create the black figure that will hold the occluder mask
                occluder_mask = np.zeros((gray.shape[0], gray.shape[1]), np.uint8)

                # drawing occluders on the image & on their mask
                for point, radius in zip(points, radiuses):
                    cv.circle(gray, point, radius, tuple(colBlob[0:3]), -1)
                    # here I draw the occluder mask
                    cv.circle(occluder_mask, point, radius, color = (255, 255, 255), thickness = -1)

                # calculate the intersection between masks
                intersection = cv.bitwise_and(object_mask, occluder_mask) # create the intersection
                intersectionSize = intersection[intersection==255].shape[0] # measure the size of intersection
                proportion_occluded = ((intersectionSize/size_object)*100)

                # write the image only if occlusion is as required
                if abs(portionVisible - (100 - proportion_occluded)) < 1:
                    cv.imwrite(writePath + '_' + str(level) + '/blob{}'.format(level) + name + '.png', gray)
                    print('Object occluded successfully')
                    break
                elif portionVisible > (100 - proportion_occluded):
                    #nOccluders += nOccluders
                    print('Occlusion is too low, the difference is', 
                          abs(portionVisible - (100 - proportion_occluded)))
                    continue
                elif portionVisible < (100 - proportion_occluded):
                    #nOccluders -= nOccluders
                    print('Occlusion is too high, the difference is', 
                          abs(portionVisible - (100 - proportion_occluded)))
                    continue

else:
    print('Deletion is off. Change parameters in the preamble to run it.')

Occlusion is too high, the difference is 21.654355545188295
Occlusion is too low, the difference is 63.37359286041348
Occlusion is too high, the difference is 17.986937252157688
Occlusion is too low, the difference is 8.186938980034384
Occlusion is too low, the difference is 49.002842357169385
Occlusion is too low, the difference is 9.393860854089453
Occlusion is too low, the difference is 28.5456461826884
Occlusion is too low, the difference is 41.39154549931317
Occlusion is too high, the difference is 15.0296762823005
Occlusion is too high, the difference is 5.476246015084371
Occlusion is too low, the difference is 70.0
Occlusion is too high, the difference is 27.346845329117315
Occlusion is too low, the difference is 53.91692368832561
Occlusion is too low, the difference is 67.77363087370085
Occlusion is too low, the difference is 43.976449040596464
Occlusion is too high, the difference is 27.491986971809695
Occlusion is too high, the difference is 20.671193703617305
Occlusion is to

Occlusion is too low, the difference is 29.39830123841017
Occlusion is too low, the difference is 40.59082034767698
Occlusion is too low, the difference is 40.920774888874476
Occlusion is too low, the difference is 27.95961298781762
Occlusion is too high, the difference is 29.61529317685708
Occlusion is too low, the difference is 69.37971427953921
Occlusion is too low, the difference is 60.33910393568047
Occlusion is too high, the difference is 29.998559150475117
Occlusion is too high, the difference is 29.796119792229504
Occlusion is too low, the difference is 42.822696261715905
Occlusion is too low, the difference is 20.173262155366807
Occlusion is too low, the difference is 38.096709820109936
Occlusion is too high, the difference is 10.104749760458759
Occlusion is too low, the difference is 34.53276852031958
Object occluded successfully
Occlusion is too low, the difference is 1.7190055256579342
Occlusion is too low, the difference is 12.587333491826783
Occlusion is too high, the dif

Object occluded successfully
Occlusion is too low, the difference is 14.66105459312466
Occlusion is too low, the difference is 3.225642192392286
Occlusion is too high, the difference is 7.884746644131695
Occlusion is too high, the difference is 36.56590936176207
Occlusion is too low, the difference is 21.588390158286487
Occlusion is too high, the difference is 39.78039241178318
Occlusion is too low, the difference is 11.291643644164196
Occlusion is too high, the difference is 3.6291047767629863
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 32.41210821117865
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 38.17044235707088
Occlusion is too high, the difference is 7.014766903933868
Occlusion is too low, the difference is 20.93725961798897
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 32.753382953597466
Occlusion is too high, the difference is 56.276421707240445
Occlusi

Occlusion is too high, the difference is 30.045703045905526
Object occluded successfully
Occlusion is too low, the difference is 55.4569091598505
Occlusion is too low, the difference is 56.40972861708467
Occlusion is too low, the difference is 11.291370998320787
Occlusion is too low, the difference is 36.07280212339526
Occlusion is too low, the difference is 29.578029359189642
Occlusion is too high, the difference is 17.724391961432204
Occlusion is too high, the difference is 3.8898217864687723
Occlusion is too low, the difference is 62.92887709224851
Occlusion is too low, the difference is 35.26677861437625
Occlusion is too low, the difference is 38.90092627701641
Occlusion is too high, the difference is 29.23676940577434
Occlusion is too low, the difference is 27.954065326905365
Occlusion is too low, the difference is 26.55706624776556
Occlusion is too low, the difference is 5.064731054655766
Occlusion is too low, the difference is 9.187476301392124
Occlusion is too low, the differen

Occlusion is too high, the difference is 25.341798413782243
Occlusion is too low, the difference is 31.328063448710544
Occlusion is too high, the difference is 6.60722727033982
Occlusion is too low, the difference is 8.661431797888547
Occlusion is too high, the difference is 2.1512421870896503
Occlusion is too high, the difference is 10.270760018908561
Occlusion is too low, the difference is 39.911760071432326
Occlusion is too high, the difference is 8.49283050580388
Occlusion is too high, the difference is 2.0862440254215073
Occlusion is too low, the difference is 20.36635327485687
Occlusion is too low, the difference is 22.967592835758182
Occlusion is too low, the difference is 52.057881191239034
Occlusion is too low, the difference is 43.5562529544619
Occlusion is too low, the difference is 9.665948841850941
Occlusion is too low, the difference is 29.426046536057562
Occlusion is too low, the difference is 34.183386732496444
Occlusion is too high, the difference is 12.11565733494406


...

Occlusion is too high, the difference is 25.713286713286713
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 56.86713286713287
Occlusion is too high, the difference is 37.53146853146853
Occlusion is too high, the difference is 67.20979020979021
Occlusion is too high, the difference is 56.16783216783216
Occlusion is too low, the difference is 18.391608391608386
Occlusion is too high, the difference is 60.69930069930069
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 7.02097902097902
Occlusion is too low, the difference is 24.88111888111888
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 18.377622377622373
Occlusion is too high, the difference is 16.5944055944056
Occlusion is too high, the difference is 70.0
Occlusion is too low, the difference is 1.0139860139860133
Occlusion is too low, the difference is 28.293706293706293
Occlusion is too low, the difference is 13.3356643

Occlusion is too low, the difference is 67.82076940182344
Occlusion is too high, the difference is 8.496775628196573
Occlusion is too high, the difference is 30.0
Occlusion is too high, the difference is 20.363946334593436
Occlusion is too low, the difference is 6.2982729226892005
Occlusion is too low, the difference is 27.03802535023349
Occlusion is too low, the difference is 70.0
Occlusion is too high, the difference is 26.9905863168038
Occlusion is too high, the difference is 30.0
Occlusion is too low, the difference is 67.28708027573938
Occlusion is too high, the difference is 16.250092654362163
Occlusion is too low, the difference is 35.18419687198873
Occlusion is too low, the difference is 61.77970498851086
Occlusion is too low, the difference is 24.19909569342525
Occlusion is too high, the difference is 27.272255577792606
Occlusion is too high, the difference is 16.79119412941961
Occlusion is too high, the difference is 30.0
Occlusion is too high, the difference is 30.0
Occlusio

Occlusion is too high, the difference is 13.06574753539396
Occlusion is too high, the difference is 39.631606256022536
Occlusion is too low, the difference is 17.880809428507888
Occlusion is too high, the difference is 37.71922022088799
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 46.78452301534357
Occlusion is too low, the difference is 2.60395819435179
Occlusion is too high, the difference is 68.39152027277444
Occlusion is too low, the difference is 16.76895708249944
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 58.15506634052332
Occlusion is too high, the difference is 20.492921206730415
Occlusion is too high, the difference is 57.962345267215184
Occlusion is too low, the difference is 12.595804610481053
Occlusion is too high, the difference is 60.378771032540214
Occlusion is too high, the difference is 42.59654

Occlusion is too low, the difference is 59.526686807653576
Occlusion is too high, the difference is 15.442094662638468
Occlusion is too low, the difference is 3.389728096676734
Occlusion is too high, the difference is 7.639476334340387
Occlusion is too low, the difference is 70.0
Occlusion is too high, the difference is 4.78348439073514
Occlusion is too low, the difference is 70.0
Occlusion is too high, the difference is 8.70694864048339
Occlusion is too low, the difference is 32.27995971802618
Occlusion is too high, the difference is 29.907351460221548
Occlusion is too high, the difference is 13.556898288016114
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 40.630412890231625
Occlusion is too low, the difference is 31.740181268882175
Occlusion is too low, the difference is 70.0
Occlusion is too low, the difference is 58.09264853977845
Occlusion is too high, the difference is 9.282980866062431
Occlusion is too low, the difference is 45.568982880161

Occlusion is too high, the difference is 62.28367391102729
Occlusion is too high, the difference is 62.014715429560695
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 45.48768046495812
Occlusion is too high, the difference is 15.234488515163697
Occlusion is too high, the difference is 58.97888521346647
Occlusion is too low, the difference is 14.944507991467518
Occlusion is too high, the difference is 63.399697035273746
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 32.92391875598974
Occlusion is too high, the difference is 67.65975206356076
Occlusion is too high, the difference is 70.0
Occlusion is too high, the difference is 8.260116857822979
Occlusion is too low, the difference is 30.0
Occlusion is too high, the difference is 29.44909883451325
Occlusion is too high, the difference is 35.69079049061736
Object occluded successfully
Occlusion is too low, the difference is 28.69858872264605
Occlusion is too low